In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

How great is this that an MNIST-like dataset is available in Kannada Language! I was absolutely delighted when I came across it... I sincerely thank the dataset author [Vinay Uday Prabhu](https://www.kaggle.com/higgstachyon/datasets) sir for making such a creative and interesting dataset available to everyone.

# Reading the Data

In [ ]:
test = pd.read_csv("../input/Kannada-MNIST/test.csv")
data = pd.read_csv("../input/Kannada-MNIST/train.csv")

In [ ]:
print(data.head())

So, we see that we have 784 pixels, that is 28x28 pixel dataset. Here are the numbers for reference and understanding :

![](https://d3i71xaburhd42.cloudfront.net/832f095f729a106fbd1cb11015851e0c86b4b805/2-TableI-1.png)

# Preparing the Data:

In [ ]:
#assigning the training data
train_y = data["label"]
train_x = data.drop(["label"], axis = 1)
#we do the same operations on the test set as well
test_y = data["label"]
test_x = data.drop(["label"], axis = 1)

#Now we convert the dataframe into an array so we can reshape the data 
train_x = np.array(train_x).reshape(-1,28,28)
test_x = np.array(test_x).reshape(-1,28,28)
#The -1 that we passed as the first parameter is the index of the last row
#So, what numpy understands is to start from Row1 till the last Row.

#reshaping the data
train_x = train_x.reshape(-1, 28,28, 1)
test_x = test_x.reshape(-1, 28,28, 1)

#We clean/normalize the data by dividing it by 255 because we want our values ranging between 0 to 1
#It also GREATLY improves the accuracy at the end
train_x = train_x/255
test_x = test_x/255

Encoding the values with "to_categorical" from keras

In [ ]:
from keras.utils.np_utils import to_categorical 
train_y = to_categorical(train_y, num_classes = 10)
test_y = to_categorical(test_y, num_classes = 10)

In [ ]:
train_x.shape, train_y.shape

# Plotting the Numbers

In [ ]:
#Displaying the images
import matplotlib.pyplot as plt
plt.figure()
for i in range(16):
    plt.subplot(4,4,i+1)
    plt.imshow(train_x[i], cmap = plt.cm.binary)
plt.show()

# Making the Neural Network

Let's make a Convolutional Neural Network for the image classification task.

Since we have 10 outputs(numbers from 0-9) we will have 10 neurons in the output layer with **softmax** as the activation function.

We have 8 layers in our CNN - Input layer, 1st Convolutional layer, 1st pooling layer, 2nd Convolutional layer, 2nd pooling layer, Flattening layer, Hidden Layer and Output layer respectively.

* If you want to read more about CNNs you can refer this link - [How Do Convolutional Layers Work in Deep Learning Neural Networks?](https://machinelearningmastery.com/convolutional-layers-for-deep-learning-neural-networks/)

* Or for more comprehensive understanding you can also refer this link [A Comprehensive Guide to Convolutional Neural Networks — the ELI5 way](https://towardsdatascience.com/a-comprehensive-guide-to-convolutional-neural-networks-the-eli5-way-3bd2b1164a53), this really cleared all my doubts regarding CNNs.

* So, in the input layer we provide the shape as a 3 dimentional tensor. The first two are the rows and columns obviously, and the third one is the colour channel, in case you're wondering, like 3 for (R,G,B), etc.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size = (3,3), activation = "linear", input_shape = (28, 28, 1), padding = "same"))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D((2,2), padding = "same"))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(10, activation='softmax'))

# Compiling the Model

The loss function used is categorical_crossentropy, the metrics used for evaluation are "accuracy", and the optimizer used is Adam. Other optimizers that you can experiment with are

*     SGD
*     RMSprop
*     Adam
*     Adadelta
*     Adagrad
*     Adamax
*     Nadam
*     Ftrl


In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
model.summary()

# Training the Model

In [ ]:
train_model = model.fit(train_x, train_y, batch_size = 64, epochs = 10)

# Predicting Test Data

In [ ]:
test_x.shape, data.shape, train_x.shape

In [ ]:
#We split the training set provided into a validation set as well to determine overfitting or underfitting to evaluate the model
#I have not done the evaluation step here, we'll just move on to prediction
predicted_value = model.predict_classes(test_x)

In [ ]:
test_x.shape

In [ ]:
#our test set has 60000 rows, but does not include an ID column, so we will define it
imageid = list(range(1,60001))
submission = pd.DataFrame({"id": imageid, "label": predicted_value})
submission.to_csv("submission.csv", index = False)